In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import folium
import geopandas

df = pd.read_csv("data/consolidation-etalab-schema-irve-statique-v-2.2.0-20230207.csv")

df_short= (
    df
    .loc[:,["id_station_local","nom_station", "code_insee_commune", "nbre_pdc", "gratuit", "consolidated_longitude", "consolidated_latitude"]]
)


geo_data = geopandas.read_file("data\communes-version-simplifiee.geojson")


df_short = (
    df
    .loc[:,["id_station_local","nom_station", "code_insee_commune", "nbre_pdc", "gratuit", "consolidated_longitude", "consolidated_latitude"]]
    .dropna(subset = ["nbre_pdc", "code_insee_commune"])
    .assign(code_insee_commune = lambda df_: df_.code_insee_commune.str.replace(".0","", regex = False))
    .groupby(by = "code_insee_commune")
    .agg({"nbre_pdc": np.sum, 
          "consolidated_longitude":"first",
          "consolidated_latitude":"first"})

    .reset_index()

)

df_shortGeometry = df_short.merge(
    geo_data,
    left_on = "code_insee_commune",
    right_on = "code"
).dropna(subset =["geometry"])

df_shortGeometry = geopandas.GeoDataFrame(df_shortGeometry)
df_shortGeometry ["logpdc"]=np.log(df_shortGeometry.nbre_pdc)

Carte de la france représantant le nombre de borne Electrique par commune


In [ ]:
fig=px.choropleth_mapbox(df_shortGeometry,
                           geojson=df_shortGeometry.geometry,
                           locations=df_shortGeometry.index,
                           color="logpdc",
                           center={"lat": 45.5517, "lon": 5},
                           mapbox_style="open-street-map",
                           zoom=5,
                           color_continuous_scale=["wheat","burlywood","orangered","red","darkred","purple"],
                           hover_name = "nom",
                           hover_data=['nbre_pdc'],
                           title= "Carte de la france représantant le nombre de borne Electrique par commune",
                           labels={'nbre_pdc': 'Borne par ville'}
                           )
fig.update_layout(legend_title_text="sample data")
fig.show()